# A Jupyter Notebook to Create a Timetable
This notebook generates a timetable based on department, year level, and other constraints.

## Import Required Libraries
Import libraries such as pandas and random for data manipulation and randomization.

In [ ]:
# Import Required Libraries
import pandas as pd
import random

## Load and Inspect Data
Load the CSV files for rooms, sections, subjects, and timetable into pandas DataFrames and inspect their structure.

In [ ]:
# Load and Inspect Data
rooms_df = pd.read_csv(r'c:\xampp\htdocs\SMS\sms3_rooms.csv')
sections_df = pd.read_csv(r'c:\xampp\htdocs\SMS\sms3_sections.csv')
subjects_df = pd.read_csv(r'c:\xampp\htdocs\SMS\sms3_subjects.csv')
timetable_df = pd.read_csv(r'c:\xampp\htdocs\SMS\sms3_timetable.csv')

# Display the first few rows of each DataFrame
print("Rooms Data:")
print(rooms_df.head())
print("\nSections Data:")
print(sections_df.head())
print("\nSubjects Data:")
print(subjects_df.head())
print("\nTimetable Data:")
print(timetable_df.head())

## Filter Data by Department and Year Level
Filter the data to ensure that subject_id, section_id, and room_id have the same department_id and match the year level constraints.

In [ ]:
# Filter Data by Department and Year Level
department_id = 1  # Example department
year_level = 1     # Example year level

# Filter subjects, sections, and rooms based on department_id and year_level
filtered_subjects = subjects_df[(subjects_df['department_id'] == department_id) & (subjects_df['year_level'] == year_level)]
filtered_sections = sections_df[(sections_df['department_id'] == department_id) & (sections_df['year_level'] == year_level)]
filtered_rooms = rooms_df[rooms_df['department_id'] == department_id]

# Display filtered data
print("Filtered Subjects:")
print(filtered_subjects)
print("\nFiltered Sections:")
print(filtered_sections)
print("\nFiltered Rooms:")
print(filtered_rooms)

## Generate Timetable
Randomly assign days (Monday to Saturday) and times (06:00:00 to 21:00:00) to each section, ensuring no section has more than 3 days.

In [ ]:
# Generate Timetable
days = ["Monday", "Tuesday", "Wednesday", "Thursday", "Friday", "Saturday"]
time_slots = ["06:00:00", "07:30:00", "09:00:00", "10:30:00", "12:00:00", "13:30:00", "15:00:00", "16:30:00", "18:00:00", "19:30:00"]

generated_timetable = []

# Exclude sections with department_id 1
filtered_sections_df = sections_df[sections_df["department_id"] != 1]

# Use filtered_sections_df in the timetable generation loop
for _, section in filtered_sections_df.iterrows():
    year_level = section["year_level"]
    section_id = section["id"]
    department_id = section["department_id"]

    # Determine the number of subjects and days based on year level
    if year_level == 1 or year_level == 2:
        num_subjects = 8
        num_days = 3  # Maximum 3 days
    elif year_level == 3:
        num_subjects = 6
        num_days = random.randint(2, 3)  # 2 to 3 days
    elif year_level == 4:
        num_subjects = 3
        num_days = random.randint(1, 2)  # 1 to 2 days
    else:
        continue  # Skip sections with unsupported year levels

    # Filter subjects for the current year level and department
    filtered_subjects = subjects_df[
        (subjects_df["year_level"] == year_level) &
        (subjects_df["department_id"] == department_id)
    ]

    # Ensure enough subjects are available for sampling
    if len(filtered_subjects) < num_subjects:
        print(f"Not enough subjects for section {section_id} (Year Level {year_level}). Skipping...")
        continue

    # Randomly select the required number of subjects
    assigned_subjects = filtered_subjects.sample(num_subjects, replace=False)  # Ensure no duplicates
    assigned_days = random.sample(days, min(num_days, len(days)))  # Assign random days

    # Distribute subjects across the assigned days
    subject_distribution = [num_subjects // num_days] * num_days  # Equal distribution
    for i in range(num_subjects % num_days):  # Distribute the remainder
        subject_distribution[i] += 1

    # Determine the number of rooms to use based on year level
    if year_level in [1, 2]:
        num_rooms = random.choice([1, 2])
    elif year_level == 3:
        num_rooms = 2
    else:
        num_rooms = 1  # Default for other year levels

    # Select the specified number of rooms for this section
    available_rooms = rooms_df[rooms_df['department_id'] == department_id]
    if len(available_rooms) < num_rooms:
        print(f"Not enough rooms available for section {section_id}. Using all available rooms.")
        num_rooms = len(available_rooms)
    allowed_rooms = available_rooms.sample(num_rooms)

    subject_index = 0
    for day, subject_count in zip(assigned_days, subject_distribution):
        for _ in range(subject_count):
            if subject_index >= len(assigned_subjects):
                break
            subject = assigned_subjects.iloc[subject_index]
            # Select a random room from the allowed rooms for this section
            room = allowed_rooms.sample(1).iloc[0]
            start_time = random.choice(time_slots)
            # Calculate end time by adding 1.5 hours (90 minutes) to start time
            start_hour, start_minute, _ = map(int, start_time.split(":"))
            end_hour = start_hour + (start_minute + 90) // 60
            end_minute = (start_minute + 90) % 60
            end_time = f"{end_hour:02d}:{end_minute:02d}:00"

            generated_timetable.append({
                "subject_id": subject.id,
                "section_id": section_id,
                "room_id": room["id"],
                "day_of_week": day,
                "start_time": start_time,
                "end_time": end_time
            })
            subject_index += 1

# Convert to DataFrame
generated_timetable_df = pd.DataFrame(generated_timetable)

# Assign incremental IDs starting at 407
generated_timetable_df['id'] = range(407, 407 + len(generated_timetable_df))

# Reorder columns to place 'id' as the first column
generated_timetable_df = generated_timetable_df[['id'] + [col for col in generated_timetable_df.columns if col != 'id']]

print("Generated Timetable:")
print(generated_timetable_df)

## Check for Conflicts
Ensure there are no conflicts in the timetable, such as overlapping times for the same room or section.

In [ ]:
# Check for Conflicts
def check_conflicts(timetable):
    conflicts = []
    for _, row in timetable.iterrows():
        overlapping = timetable[
            (timetable["room_id"] == row["room_id"]) &
            (timetable["day_of_week"] == row["day_of_week"]) &
            (
                (timetable["start_time"] < row["end_time"]) &
                (timetable["end_time"] > row["start_time"])
            )
        ]
        if len(overlapping) > 1:
            conflicts.append(row)
    return conflicts

conflicts = check_conflicts(generated_timetable_df)
if conflicts:
    print("Conflicts Found:")
    print(conflicts)
else:
    print("No conflicts found in the timetable.")

## Save Timetable to CSV
Save the generated timetable to a new CSV file for further use.

In [ ]:
# Save Timetable to CSV
output_path = r'c:\xampp\htdocs\SMS\generated_timetable.csv'
generated_timetable_df.to_csv(output_path, index=False)
print(f"Generated timetable saved to {output_path}")